# Exercise 4: Calibrate and Project an SEIR Model with Interventions

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ngozzi/tech-transfer-epdemix/blob/main/sessions/session-4/exercises/python/exercise_4_seir_calibration.ipynb)

**Scenario:** An outbreak occurs in California. After some time, interventions are implemented that reduce transmission. We will:

1. Generate synthetic data from an SEIR model with an intervention (β drops mid-epidemic)
2. Calibrate the model to recover the initial β and the reduction factor
3. Project forward under different scenarios (status quo vs. relaxation)

In [ ]:
# Colab installation (skip if running locally)
import sys, os, subprocess
if "google.colab" in sys.modules or os.getenv("COLAB_RELEASE_TAG"):
    subprocess.run([sys.executable, "-m", "pip", "install", "-q", "-r",
                    "https://raw.githubusercontent.com/epistorm/epydemix/refs/heads/main/tutorials/colab_requirements.txt"])

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from datetime import datetime, timedelta

from epydemix import EpiModel, simulate
from epydemix.population import load_epydemix_population
from epydemix.calibration import ABCSampler, rmse
from epydemix.visualization import plot_quantiles, plot_posterior_distribution
from epydemix.utils import compute_simulation_dates

colors = sns.color_palette("Dark2")

## Task 1: Generate Synthetic Data

Create an SEIR model for California where:
- Initial β = 0.035 (baseline transmission)
- On day 50, interventions reduce β to 60% of original (reduction_factor = 0.6)
- The epidemic runs for 120 days total

**Hint:** Use `override_parameter` to implement the intervention.

In [ ]:
# True parameters (what we'll try to recover)
TRUE_BETA = 0.035
TRUE_REDUCTION = 0.6  # β drops to 60% of original
INTERVENTION_DAY = 50

# Simulation settings
START_DATE = "2026-01-01"
END_DATE = "2026-04-30"  # 120 days
CALIBRATION_END = "2026-03-15"  # Calibrate up to day 74

# Fixed disease parameters
SIGMA = 0.2   # 5-day latent period
GAMMA = 0.1   # 10-day infectious period

# TODO: Compute intervention date
intervention_date = ...

In [ ]:
# TODO: Create a function to build the SEIR model
def create_seir_model():
    # Create model with compartments S, E, I, R
    model = ...
    
    # Add transitions:
    # S -> E (mediated by I)
    # E -> I (spontaneous)
    # I -> R (spontaneous)
    
    # Add parameters
    
    # Load California population
    
    return model

# Create truth model and add intervention
model_truth = create_seir_model()

# TODO: Add intervention using override_parameter
# β drops to TRUE_BETA * TRUE_REDUCTION after INTERVENTION_DAY

model_truth

In [ ]:
# TODO: Set up initial conditions (seed with 1000 infected)
population = model_truth.population
n_groups = len(population.Nk)

initial_conditions = ...

# TODO: Run one simulation to generate "observed" data
results_truth = ...

# TODO: Extract incidence (E→I transitions)
incidence = ...
dates = pd.date_range(start=START_DATE, periods=len(incidence), freq="D")

# Create data DataFrame
data = pd.DataFrame({"date": dates, "data": incidence})
print(f"Generated {len(data)} days of data")
print(f"Peak incidence: {data['data'].max():.0f} on day {data['data'].argmax()}")

In [ ]:
# TODO: Split into calibration and projection periods
calibration_end_date = pd.to_datetime(CALIBRATION_END)
data_calibration = ...
data_projection = ...

print(f"Calibration: {data_calibration.date.iloc[0].date()} to {data_calibration.date.iloc[-1].date()} ({len(data_calibration)} days)")
print(f"Projection:  {data_projection.date.iloc[0].date()} to {data_projection.date.iloc[-1].date()} ({len(data_projection)} days)")

In [ ]:
# TODO: Visualize the synthetic data
# Plot calibration data (black), projection data (gray)
# Mark intervention date (blue line) and calibration cutoff (red line)

fig, ax = plt.subplots(figsize=(12, 5), dpi=150)

# Your code here

plt.tight_layout()

## Task 2: Set Up the Calibration Model

We'll calibrate two parameters:
- `beta`: Initial transmission rate (before intervention)
- `reduction`: The factor by which β is reduced after the intervention

**Hint:** The wrapper function needs to apply the intervention using the sampled parameters.

In [ ]:
# Create calibration model
model_calib = create_seir_model()

# Store end date for wrapper
OVERRIDE_END_DATE = pd.to_datetime(END_DATE)

In [ ]:
# TODO: Define wrapper function
def simulate_wrapper(parameters):
    """Wrapper that applies intervention and runs simulation."""
    # Extract sampled parameters
    reduction = parameters.get("reduction", 1.0)
    beta = parameters.get("beta", 0.035)
    
    # TODO: Clear previous overrides and add intervention
    # Hint: use parameters["epimodel"].clear_overrides()
    # Then use override_parameter with value = beta * reduction
    
    # Run simulation
    results = simulate(**parameters)
    
    return {"data": results.transitions["E_to_I_total"]}

## Task 3: Define Priors and Run Calibration

In [ ]:
# TODO: Define priors for beta and reduction
# beta: U(0.02, 0.045)
# reduction: U(0.3, 0.9)
priors = {
    "beta": ...,
    "reduction": ...
}

print("Prior distributions:")
print(f"  beta: U(0.02, 0.045)")
print(f"  reduction: U(0.3, 0.9)")
print(f"\nTrue values: beta={TRUE_BETA}, reduction={TRUE_REDUCTION}")

In [ ]:
# TODO: Set up calibration parameters and ABC sampler
calibration_parameters = {
    "initial_conditions_dict": initial_conditions,
    "epimodel": model_calib,
    "start_date": START_DATE,
    "end_date": CALIBRATION_END,
}

# Initialize ABC sampler
abc_sampler = ABCSampler(
    simulation_function=simulate_wrapper,
    priors=priors,
    parameters=calibration_parameters,
    observed_data=data_calibration["data"].values,
    distance_function=rmse
)

In [ ]:
# TODO: Run ABC-SMC calibration with 100 particles and 5 generations
print("Running calibration...")
calibration_results = ...
print("Done!")

In [ ]:
# TODO: Visualize posterior distributions
# Plot beta and reduction posteriors with true values marked
posterior = calibration_results.get_posterior_distribution()

fig, axes = plt.subplots(1, 2, figsize=(10, 4), dpi=150)

# Your code here

plt.tight_layout()

In [ ]:
# TODO: Print posterior summaries
print("Posterior estimates vs True values:")
print(f"{'Parameter':<12} {'Median':<10} {'IQR':<20} {'True':<10}")
print("-" * 52)

for param, true_val in [("beta", TRUE_BETA), ("reduction", TRUE_REDUCTION)]:
    values = posterior[param].values
    # Your code here

## Task 4: Project Under Alternative Scenarios

Compare two scenarios for the projection period:

1. **Status quo**: Interventions remain in place (β stays reduced)
2. **Relaxation**: Interventions are lifted (β returns to baseline)

**Hint:** Change `OVERRIDE_END_DATE` between projections to control when the intervention ends.

In [ ]:
# TODO: Run projections for both scenarios
print("Running scenario projections...")

# Status quo (interventions maintained)
params_status_quo = calibration_parameters.copy()
params_status_quo["end_date"] = data_projection.date.values[-1]
results_status_quo = ...
print("  Status quo: done")

# Relaxation (interventions lifted)
# Hint: Change OVERRIDE_END_DATE to CALIBRATION_END before running
OVERRIDE_END_DATE = CALIBRATION_END
params_relaxation = calibration_parameters.copy()
params_relaxation["end_date"] = data_projection.date.values[-1]
results_relaxation = ...
print("  Relaxation: done")

In [ ]:
# TODO: Visualize scenario comparison
# Get quantiles for both scenarios and plot with observed data

projection_dates = compute_simulation_dates(
    start_date=params_status_quo["start_date"],
    end_date=params_status_quo["end_date"],
)

df_status_quo = results_status_quo.get_projection_quantiles(projection_dates)
df_relaxation = results_relaxation.get_projection_quantiles(projection_dates)

fig, ax = plt.subplots(dpi=150, figsize=(12, 5))

# Your code here

plt.tight_layout()

## Discussion

*Write your observations here:*

1. **Parameter recovery**: Did the calibration recover the true values of β and the reduction factor?

2. **Scenario projections**: How do the two scenarios differ in projected infections?

3. **Policy implications**: What do these results suggest about intervention timing and relaxation strategies?